![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&file=BQML+Demo+2024.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/new/BQML%20Demo%202024.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Fnew%2FBQML%2520Demo%25202024.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/new/BQML%20Demo%202024.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/new/BQML%20Demo%202024.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# BQML DEMO 2024

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/TensorFlow/TensorFlow%20Basics%20-%20Data%20To%20Training.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('bigframes', 'bigframes'),
    ('google.cloud.bigquery_connection_v1', 'google-cloud-bigquery-connection'),
    ('plotly', 'plotly'),
    ('kaleido', 'kaleido')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

In [4]:
#!sudo apt-get -qq install graphviz

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
# BigQuery Parameters
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'bqml_2024'
BQ_TABLE_PREFIX = 'thelook'
BQ_REGION = 'us'
BQ_SOURCE_DATASET = 'bigquery-public-data.thelook_ecommerce'

packages:

In [8]:
from google.cloud import bigquery
from google.cloud import bigquery_connection_v1 as bq_connection
import bigframes.pandas as bpd
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'

clients:

In [9]:
bq = bigquery.Client(project = PROJECT_ID)
bpd.options.bigquery.project = PROJECT_ID

---
## Environment Setup

### BigQuery Dataset

In [70]:
try:
    ds = bq.get_dataset(f'{BQ_PROJECT}.{BQ_DATASET}')
    print('Found the BigQuery Dataset: ', ds.full_dataset_id)
except:
    ds = bigquery.DatasetReference(BQ_PROJECT, BQ_DATASET)
    ds.location = BQ_REGION
    ds = bq.create_dataset(dataset = ds, exists_ok = True)
    print('Created the BigQuery Dataset: ', ds.full_dataset_id)

Found the BigQuery Dataset:  statmike-mlops-349915:bqml_2024


### Connection Requirement

To make a remote connection using BigQuery ML, BigQuery uses a CLOUD_RESOURCE connection. [Reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model#connection)

Create a new connection with type `CLOUD_RESOURCE`: First, check for existing connection.

In [72]:
try:
    response = bq_connection.ConnectionServiceClient().get_connection(
            request = bq_connection.GetConnectionRequest(
                name = f"projects/{BQ_PROJECT}/locations/{BQ_REGION}/connections/{BQ_DATASET}"
            )
    )
    print(f'Found existing connection with service account: {response.cloud_resource.service_account_id}')
    service_account = response.cloud_resource.service_account_id
except Exception:
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": f"projects/{BQ_PROJECT}/locations/{BQ_REGION}",
            "connection_id": f"{BQ_DATASET}",
            "connection": bq_connection.types.Connection(
                {
                    "friendly_name": f"{BQ_DATASET}",
                    "cloud_resource": bq_connection.CloudResourceProperties({})
                }
            )
        }
    )
    response = bq_connection.ConnectionServiceClient().create_connection(request)
    print(f'Created new connection with service account: {response.cloud_resource.service_account_id}')
    service_account = response.cloud_resource.service_account_id

Created new connection with service account: bqcx-1026793852137-d2h9@gcp-sa-bigquery-condel.iam.gserviceaccount.com


Assign the service account the Vertex AI User role:

In [73]:
# for llm usage on vertex
!gcloud projects add-iam-policy-binding {BQ_PROJECT} --member=serviceAccount:{service_account} --role=roles/aiplatform.user

Updated IAM policy for project [statmike-mlops-349915].
bindings:
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-1026793852137-bmph@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-d2h9@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-dyw1@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-pdxa@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-te86@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-tqpc@gcp-sa-big

In [176]:
# for cloud nlp usage:
!gcloud projects add-iam-policy-binding {BQ_PROJECT} --member=serviceAccount:{service_account} --role=roles/serviceusage.serviceUsageConsumer

Updated IAM policy for project [statmike-mlops-349915].
bindings:
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-1026793852137-bmph@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-d2h9@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-dyw1@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-pdxa@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-te86@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-tqpc@gcp-sa-big

In [187]:
# for cloud translate usage:
!gcloud projects add-iam-policy-binding {BQ_PROJECT} --member=serviceAccount:{service_account} --role=roles/cloudtranslate.user

Updated IAM policy for project [statmike-mlops-349915].
bindings:
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-1026793852137-bmph@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-d2h9@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-dyw1@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-pdxa@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-te86@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-tqpc@gcp-sa-big

### Create The Remote Models In BigQuery

In [78]:
# Create Remote Model In BigQuery
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.VERTEX_LLM`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(ENDPOINT = 'text-bison@002')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [79]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.VERTEX_LLM`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(ENDPOINT = 'text-bison@002')



In [80]:
# Create Remote Model In BigQuery
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_TRANSLATE`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_TRANSLATE_V3')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [81]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.CLOUD_TRANSLATE`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_TRANSLATE_V3')



In [82]:
# Create Remote Model In BigQuery
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_NLP`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_NATURAL_LANGUAGE_V1')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [83]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.CLOUD_NLP`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_NATURAL_LANGUAGE_V1')



---
## Intro & Explore

In [20]:
query = f'''
# 1
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts` AS
WITH
    TOP5 AS (
        SELECT product_id, count(*) as total
        FROM `{BQ_SOURCE_DATASET}.order_items`
        GROUP BY product_id
    )
SELECT
    t.product_id,
    t.total,
    p.category,
    p.department,
    p.name
FROM TOP5 as t
JOIN `{BQ_SOURCE_DATASET}.products` as p on p.id = t.product_id
WHERE p.department = 'Men'
ORDER by t.total DESC
LIMIT 5
'''
print(query)


# 1
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_topMensProducts` AS
WITH
    TOP5 AS (
        SELECT product_id, count(*) as total
        FROM `bigquery-public-data.thelook_ecommerce.order_items`
        GROUP BY product_id
    )
SELECT
    t.product_id,
    t.total,
    p.category,
    p.department,
    p.name
FROM TOP5 as t
JOIN `bigquery-public-data.thelook_ecommerce.products` as p on p.id = t.product_id
WHERE p.department = 'Men'
ORDER by t.total DESC
LIMIT 5



In [21]:
job = bq.query(query = query)
job.result()

In [22]:
# Create Remote Model In BigQuery
query = f"""
# 2.1
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.VERTEX_LLM`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(ENDPOINT = 'text-bison@002')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [23]:
print(query)


# 2.1
CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.VERTEX_LLM`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(ENDPOINT = 'text-bison@002')



In [24]:
query = f'''
# 2.2
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts` AS
SELECT
    * EXCEPT(ml_generate_text_llm_result, ml_generate_text_status),
    ml_generate_text_llm_result as description
FROM ML.GENERATE_TEXT(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.VERTEX_LLM`,
    (SELECT *,
        CONCAT(
            'Create a descriptive paragraph of no more than 25 words for a product with in a department named ', department,
            ', category named "', category, '"',
            'and the following description: ', name
        ) as prompt
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts`),
    STRUCT(200 AS max_output_tokens, TRUE AS flatten_json_output)
)
'''
print(query)


# 2.2
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_topMensProducts` AS
SELECT
    * EXCEPT(ml_generate_text_llm_result, ml_generate_text_status),
    ml_generate_text_llm_result as description
FROM ML.GENERATE_TEXT(
    MODEL `statmike-mlops-349915.bqml_2024.VERTEX_LLM`,
    (SELECT *,
        CONCAT(
            'Create a descriptive paragraph of no more than 25 words for a product with in a department named ', department,
            ', category named "', category, '"',
            'and the following description: ', name
        ) as prompt
        FROM `statmike-mlops-349915.bqml_2024.thelook_topMensProducts`),
    STRUCT(200 AS max_output_tokens, TRUE AS flatten_json_output)
)



In [25]:
job = bq.query(query = query)
job.result()

In [26]:
# Create Remote Model In BigQuery
query = f"""
# 3.1
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_NLP`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_NATURAL_LANGUAGE_V1')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [27]:
print(query)


# 3.1
CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.CLOUD_NLP`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_NATURAL_LANGUAGE_V1')



In [28]:
query = f'''
# 3.2
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts` AS
SELECT
    * EXCEPT(text_content, ml_understand_text_result, ml_understand_text_status),
    ml_understand_text_result.document_sentiment.score as sentiment_score
FROM ML.UNDERSTAND_TEXT(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_NLP`,
    (SELECT *, description as text_content
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts`),
    STRUCT('ANALYZE_SENTIMENT' AS nlu_option)
)
'''
print(query)


# 3.2
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_topMensProducts` AS
SELECT
    * EXCEPT(text_content, ml_understand_text_result, ml_understand_text_status),
    ml_understand_text_result.document_sentiment.score as sentiment_score
FROM ML.UNDERSTAND_TEXT(
    MODEL `statmike-mlops-349915.bqml_2024.CLOUD_NLP`,
    (SELECT *, description as text_content
        FROM `statmike-mlops-349915.bqml_2024.thelook_topMensProducts`),
    STRUCT('ANALYZE_SENTIMENT' AS nlu_option)
)



In [29]:
job = bq.query(query = query)
job.result()

In [30]:
# Create Remote Model In BigQuery
query = f"""
# 4.1
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_TRANSLATE`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_TRANSLATE_V3')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [31]:
print(query)


# 4.1
CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.CLOUD_TRANSLATE`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_TRANSLATE_V3')



In [32]:
query = f'''
# 4.2
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts` AS
SELECT
    * EXCEPT(text_content, ml_translate_result, ml_translate_status),
    ml_translate_result.translations[0].translated_text
FROM ML.TRANSLATE(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_TRANSLATE`,
    (SELECT *, description as text_content
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts`),
    STRUCT('TRANSLATE_TEXT' AS translate_mode, 'es' AS target_language_code)
)
'''
print(query)


# 4.2
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_topMensProducts` AS
SELECT
    * EXCEPT(text_content, ml_translate_result, ml_translate_status),
    ml_translate_result.translations[0].translated_text
FROM ML.TRANSLATE(
    MODEL `statmike-mlops-349915.bqml_2024.CLOUD_TRANSLATE`,
    (SELECT *, description as text_content
        FROM `statmike-mlops-349915.bqml_2024.thelook_topMensProducts`),
    STRUCT('TRANSLATE_TEXT' AS translate_mode, 'es' AS target_language_code)
)



In [33]:
job = bq.query(query = query)
job.result()

---
## Prepare Data

In [34]:
query = f'''
# 5
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_raw` AS (
    SELECT
        DATE(o.created_at) as order_date,
        o.order_id as order_id,
        o.user_id AS customer_id,
        oi.sale_price as unit_price,
        oi.product_id as product_id,
        oi.sale_price as revenue,
        u.age as age,
        u.country as country,
        u.gender as gender
    FROM `{BQ_SOURCE_DATASET}.orders` as o
    JOIN `{BQ_SOURCE_DATASET}.order_items` as oi ON o.user_id = oi.user_id
    JOIN `{BQ_SOURCE_DATASET}.users` as u ON o.user_id = u.id
    #WHERE DATE(o.created_at) < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
);
'''

In [35]:
print(query)


# 5
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_raw` AS (
    SELECT
        DATE(o.created_at) as order_date,
        o.order_id as order_id,
        o.user_id AS customer_id,
        oi.sale_price as unit_price,
        oi.product_id as product_id,
        oi.sale_price as revenue,
        u.age as age,
        u.country as country,
        u.gender as gender
    FROM `bigquery-public-data.thelook_ecommerce.orders` as o
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` as oi ON o.user_id = oi.user_id
    JOIN `bigquery-public-data.thelook_ecommerce.users` as u ON o.user_id = u.id
    #WHERE DATE(o.created_at) < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
);



In [36]:
job = bq.query(query = query)
job.result()

In [37]:
query = f"""
# 6
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features` AS
WITH
    get_rfm AS (
        SELECT
            customer_id, age, gender, country,
            DATE_DIFF(DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH), MAX(order_date), DAY) AS recency,
            COUNT(order_id) AS frequency,
            SUM(revenue) AS monetary,
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_raw`
        WHERE order_date < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
        GROUP BY customer_id, age, gender, country
    ),
    get_quartiles AS (
        SELECT
            customer_id,
            NTILE(4) OVER (ORDER BY recency ASC) AS recency_quartile,
            NTILE(4) OVER (ORDER BY frequency DESC) AS frequency_quartile,
            NTILE(4) OVER (ORDER BY monetary DESC) AS monetary_quartile,
        FROM get_rfm
    ),
    get_rfm_score AS (
        SELECT
            customer_id,
            recency_quartile + frequency_quartile + monetary_quartile AS rfm_score
        FROM get_quartiles
    )
SELECT
    CAST(rfm.customer_id AS STRING) AS customer_id,
    rfm.age,
    rfm.gender,
    rfm.country,
    rfm.recency,
    rfm.frequency,
    rfm.monetary,
    q.recency_quartile,
    q.frequency_quartile,
    q.monetary_quartile,
    rfms.rfm_score
FROM get_rfm as rfm
LEFT JOIN get_quartiles as q ON rfm.customer_id = q.customer_id
LEFT JOIN get_rfm_score as rfms ON rfm.customer_id = rfms.customer_id
"""

In [38]:
print(query)


# 6
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_features` AS
WITH
    get_rfm AS (
        SELECT
            customer_id, age, gender, country,
            DATE_DIFF(DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH), MAX(order_date), DAY) AS recency,
            COUNT(order_id) AS frequency,
            SUM(revenue) AS monetary,
        FROM `statmike-mlops-349915.bqml_2024.thelook_raw`
        WHERE order_date < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
        GROUP BY customer_id, age, gender, country
    ),
    get_quartiles AS (
        SELECT
            customer_id,
            NTILE(4) OVER (ORDER BY recency ASC) AS recency_quartile,
            NTILE(4) OVER (ORDER BY frequency DESC) AS frequency_quartile,
            NTILE(4) OVER (ORDER BY monetary DESC) AS monetary_quartile,
        FROM get_rfm
    ),
    get_rfm_score AS (
        SELECT
            customer_id,
            recency_quartile + frequency_quartile + monetary_quartile AS rfm_score
        F

In [39]:
job = bq.query(query = query)
job.result()

---
## Customer Segmentation

In [40]:
query = f'''
# 7.1
CREATE MODEL IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans8`
OPTIONS (
    # model specs
        model_type = 'KMEANS',
        num_clusters = 8,
        kmeans_init_method = 'KMEANS++',
        distance_type = 'EUCLIDEAN', 
        standardize_features = TRUE,
        
    # training specs
        max_iterations = 25,
        early_stop = TRUE,
        min_rel_progress = 0.005,
    ) AS
SELECT * EXCEPT(customer_id)
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features`
'''

In [41]:
print(query)


# 7.1
CREATE MODEL IF NOT EXISTS `statmike-mlops-349915.bqml_2024.thelook_features_kmeans8`
OPTIONS (
    # model specs
        model_type = 'KMEANS',
        num_clusters = 8,
        kmeans_init_method = 'KMEANS++',
        distance_type = 'EUCLIDEAN', 
        standardize_features = TRUE,
        
    # training specs
        max_iterations = 25,
        early_stop = TRUE,
        min_rel_progress = 0.005,
    ) AS
SELECT * EXCEPT(customer_id)
FROM `statmike-mlops-349915.bqml_2024.thelook_features`



In [42]:
query = f'''
# 7
CREATE MODEL IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`
OPTIONS (
    # model specs
        model_type = 'KMEANS',
        num_clusters = HPARAM_RANGE(2, 20),
        kmeans_init_method = 'KMEANS++',
        distance_type = 'EUCLIDEAN', 
        standardize_features = TRUE,
        
    # training specs
        max_iterations = 25,
        early_stop = TRUE,
        min_rel_progress = 0.005,
        
    # hyperparameter specs
        hparam_tuning_algorithm = 'VIZIER_DEFAULT',
        hparam_tuning_objectives = ['davies_bouldin_index'],
        num_trials = 12,
        max_parallel_trials = 3
    ) AS
SELECT * EXCEPT(customer_id)
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features`
'''

In [43]:
print(query)


# 7
CREATE MODEL IF NOT EXISTS `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`
OPTIONS (
    # model specs
        model_type = 'KMEANS',
        num_clusters = HPARAM_RANGE(2, 20),
        kmeans_init_method = 'KMEANS++',
        distance_type = 'EUCLIDEAN', 
        standardize_features = TRUE,
        
    # training specs
        max_iterations = 25,
        early_stop = TRUE,
        min_rel_progress = 0.005,
        
    # hyperparameter specs
        hparam_tuning_algorithm = 'VIZIER_DEFAULT',
        hparam_tuning_objectives = ['davies_bouldin_index'],
        num_trials = 12,
        max_parallel_trials = 3
    ) AS
SELECT * EXCEPT(customer_id)
FROM `statmike-mlops-349915.bqml_2024.thelook_features`



In [25]:
job = bq.query(query = query)
job.result()

In [44]:
query = f'''
# 8
SELECT *
FROM ML.PREDICT (
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`,
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features` LIMIT 5)
)
'''

In [45]:
print(query)


# 8
SELECT *
FROM ML.PREDICT (
    MODEL `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`,
    (SELECT * FROM `statmike-mlops-349915.bqml_2024.thelook_features` LIMIT 5)
)



In [46]:
job = bq.query(query = query)
job.result()
job.to_dataframe()

,trial_id,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,customer_id,age,gender,country,recency,frequency,monetary,recency_quartile,frequency_quartile,monetary_quartile,rfm_score
0,9,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.831333153719...",81911,56,M,Brasil,256,12,1039.350002,2,1,1,4
1,9,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.717897288056...",75461,50,M,China,256,12,1302.959995,2,1,1,4
2,9,5,"[{'CENTROID_ID': 5, 'DISTANCE': 1.659492428402...",38935,54,M,Germany,256,5,319.400005,2,1,1,4
3,9,3,"[{'CENTROID_ID': 3, 'DISTANCE': 3.905680166403...",42582,14,M,Belgium,256,40,1833.479988,2,1,1,4
4,9,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.927363825356...",96347,63,F,United States,256,18,762.510000,2,1,1,4


In [47]:
query = f'''
# 9
SELECT *
FROM (
    SELECT *
    FROM ML.DETECT_ANOMALIES (
        MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`,
        STRUCT(0.01 AS contamination),
        (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features`)
    )
)
WHERE is_anomaly = TRUE
ORDER BY normalized_distance DESC
'''
print(query)


# 9
SELECT *
FROM (
    SELECT *
    FROM ML.DETECT_ANOMALIES (
        MODEL `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`,
        STRUCT(0.01 AS contamination),
        (SELECT * FROM `statmike-mlops-349915.bqml_2024.thelook_features`)
    )
)
WHERE is_anomaly = TRUE
ORDER BY normalized_distance DESC



In [48]:
job = bq.query(query = query)
job.result()
job.to_dataframe()

,trial_id,is_anomaly,normalized_distance,CENTROID_ID,customer_id,age,gender,country,recency,frequency,monetary,recency_quartile,frequency_quartile,monetary_quartile,rfm_score
0,9,True,4.322382,3,99637,60,M,Brasil,12,32,7105.599991,1,1,1,3
1,9,True,2.960476,3,59803,29,M,China,303,36,5511.760002,3,1,1,5
2,9,True,2.531978,3,98828,16,M,Brasil,171,32,5051.959991,2,1,1,4
3,9,True,2.312755,3,91255,32,M,United States,66,20,4843.760002,1,1,1,3
4,9,True,2.128843,3,39304,35,M,China,39,36,4545.560005,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,9,True,1.411526,8,82931,44,M,Belgium,863,1,300.000000,4,4,1,8
414,9,True,1.411381,1,35173,37,M,United States,376,1,374.950012,3,4,1,8
415,9,True,1.410972,1,84937,55,M,China,1212,8,931.159998,4,1,1,6
416,9,True,1.410455,1,4269,62,M,United States,1030,14,670.540005,4,1,1,6


In [49]:
query = f'''
# 10
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_segmentSummary` AS
WITH
    SEGMENTS AS (
        SELECT * EXCEPT(TRIAL_ID, NEAREST_CENTROIDS_DISTANCE)
        FROM ML.PREDICT (
            MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`,
            (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features`)
        )
    )
SELECT
    CENTROID_ID,
    100 * (count(*) / sum(count(*)) OVER()) AS pct,
    AVG(age) AS avg_age,
    APPROX_TOP_COUNT(gender, 1)[OFFSET(0)].value AS mode_gender,
    APPROX_TOP_COUNT(country, 1)[OFFSET(0)].value AS first_country,
    APPROX_TOP_COUNT(country, 2)[OFFSET(1)].value AS second_country,
    APPROX_TOP_COUNT(country, 3)[OFFSET(2)].value AS third_country,
    APPROX_TOP_COUNT(country, 4)[OFFSET(3)].value AS fourth_country,
    AVG(recency) AS avg_recency,
    AVG(frequency) AS avg_frequency,
    AVG(monetary) AS avg_monetary,
    APPROX_TOP_COUNT(recency_quartile, 1)[OFFSET(0)].value AS mode_recency_quartile,
    APPROX_TOP_COUNT(frequency_quartile, 1)[OFFSET(0)].value AS mode_frequency_quartile,
    APPROX_TOP_COUNT(monetary_quartile, 1)[OFFSET(0)].value AS mode_monetary_quartile,
    AVG(rfm_score) as avg_rfm_score
FROM SEGMENTS
GROUP BY CENTROID_ID
ORDER BY CENTROID_ID
'''

In [50]:
print(query)


# 10
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_segmentSummary` AS
WITH
    SEGMENTS AS (
        SELECT * EXCEPT(TRIAL_ID, NEAREST_CENTROIDS_DISTANCE)
        FROM ML.PREDICT (
            MODEL `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`,
            (SELECT * FROM `statmike-mlops-349915.bqml_2024.thelook_features`)
        )
    )
SELECT
    CENTROID_ID,
    100 * (count(*) / sum(count(*)) OVER()) AS pct,
    AVG(age) AS avg_age,
    APPROX_TOP_COUNT(gender, 1)[OFFSET(0)].value AS mode_gender,
    APPROX_TOP_COUNT(country, 1)[OFFSET(0)].value AS first_country,
    APPROX_TOP_COUNT(country, 2)[OFFSET(1)].value AS second_country,
    APPROX_TOP_COUNT(country, 3)[OFFSET(2)].value AS third_country,
    APPROX_TOP_COUNT(country, 4)[OFFSET(3)].value AS fourth_country,
    AVG(recency) AS avg_recency,
    AVG(frequency) AS avg_frequency,
    AVG(monetary) AS avg_monetary,
    APPROX_TOP_COUNT(recency_quartile, 1)[OFFSET(0)].value AS mode_recency_quartil

In [51]:
job = bq.query(query = query)
job.result()

In [52]:
query = f'''
# 11
WITH
    SHOTS AS (
        SELECT
            CONCAT(
                'Cluster ', CAST(CENTROID_ID AS STRING), ', ',
                CAST(ROUND(pct, 2) AS STRING), ' percent of customers, ',
                CAST(ROUND(avg_age, 2) AS STRING), ' average age of customers, ',
                CAST(ROUND(avg_recency, 2) AS STRING), ' average days since order, ',
                CAST(ROUND(avg_frequency, 2) AS STRING), ' average total orders, ', 
                CAST(ROUND(avg_monetary, 2) AS STRING), ' average total spend, ',
                CAST(mode_recency_quartile AS STRING), ' most common quartile for days since order, ',
                CAST(mode_frequency_quartile AS STRING), ' most common quartile for total orders, ',
                CAST(mode_monetary_quartile AS STRING), ' most common quartile for total spend, ',
                'most common countries: ', first_country, ', ', second_country, ', ', third_country, ', ', fourth_country,
                '.'
            ) as shot
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_segmentSummary`
    ),
    PROMPT AS (
        SELECT
            CONCAT(
                'The customers for your retail store have be grouped in clusters. ',
                'Create a description for each of these clusters:',
                '\\n\\n',
                STRING_AGG(shot, '\\n')
            ) as prompt
        FROM SHOTS
    )
SELECT *
FROM PROMPT
'''

In [53]:
print(query)


# 11
WITH
    SHOTS AS (
        SELECT
            CONCAT(
                'Cluster ', CAST(CENTROID_ID AS STRING), ', ',
                CAST(ROUND(pct, 2) AS STRING), ' percent of customers, ',
                CAST(ROUND(avg_age, 2) AS STRING), ' average age of customers, ',
                CAST(ROUND(avg_recency, 2) AS STRING), ' average days since order, ',
                CAST(ROUND(avg_frequency, 2) AS STRING), ' average total orders, ', 
                CAST(ROUND(avg_monetary, 2) AS STRING), ' average total spend, ',
                CAST(mode_recency_quartile AS STRING), ' most common quartile for days since order, ',
                CAST(mode_frequency_quartile AS STRING), ' most common quartile for total orders, ',
                CAST(mode_monetary_quartile AS STRING), ' most common quartile for total spend, ',
                'most common countries: ', first_country, ', ', second_country, ', ', third_country, ', ', fourth_country,
                '.'
            ) as shot

In [54]:
job = bq.query(query = query)
job.result()
result = job.to_dataframe()
result

,prompt
0,The customers for your retail store have be gr...


In [55]:
print(result['prompt'].iloc[0])

The customers for your retail store have be grouped in clusters. Create a description for each of these clusters:

Cluster 1, 13.8 percent of customers, 42.71 average age of customers, 619.2 average days since order, 5.35 average total orders, 334.06 average total spend, 4 most common quartile for days since order, 1 most common quartile for total orders, 1 most common quartile for total spend, most common countries: China, United States, Brasil, South Korea.
Cluster 2, 13.68 percent of customers, 27.11 average age of customers, 188.05 average days since order, 2.89 average total orders, 158.87 average total spend, 2 most common quartile for days since order, 2 most common quartile for total orders, 2 most common quartile for total spend, most common countries: China, United States, Brasil, South Korea.
Cluster 3, 0.99 percent of customers, 41.88 average age of customers, 229.6 average days since order, 25.33 average total orders, 2321.59 average total spend, 1 most common quartile for

In [56]:
query = f'''
# 12
WITH
    SHOTS AS (
        SELECT
            CONCAT(
                'Cluster ', CAST(CENTROID_ID AS STRING), ', ',
                CAST(ROUND(pct, 2) AS STRING), ' percent of customers, ',
                CAST(ROUND(avg_age, 2) AS STRING), ' average age of customers, ',
                CAST(ROUND(avg_recency, 2) AS STRING), ' average days since order, ',
                CAST(ROUND(avg_frequency, 2) AS STRING), ' average total orders, ', 
                CAST(ROUND(avg_monetary, 2) AS STRING), ' average total spend, ',
                CAST(mode_recency_quartile AS STRING), ' most common quartile for days since order, ',
                CAST(mode_frequency_quartile AS STRING), ' most common quartile for total orders, ',
                CAST(mode_monetary_quartile AS STRING), ' most common quartile for total spend, ',
                'most common countries: ', first_country, ', ', second_country, ', ', third_country, ', ', fourth_country,
                '.'
            ) as shot
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_segmentSummary`
    ),
    PROMPT AS (
        SELECT
            CONCAT(
                'The customers for your retail store have be grouped in clusters. ',
                'Create a brief description for each of these clusters:',
                '\\n\\n',
                STRING_AGG(shot, '\\n')
            ) as prompt
        FROM SHOTS
    )
SELECT
    ml_generate_text_llm_result as response
FROM ML.GENERATE_TEXT(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.VERTEX_LLM`,
    (SELECT * FROM PROMPT),
    STRUCT(1000 AS max_output_tokens, TRUE AS flatten_json_output)
)
'''
print(query)


# 12
WITH
    SHOTS AS (
        SELECT
            CONCAT(
                'Cluster ', CAST(CENTROID_ID AS STRING), ', ',
                CAST(ROUND(pct, 2) AS STRING), ' percent of customers, ',
                CAST(ROUND(avg_age, 2) AS STRING), ' average age of customers, ',
                CAST(ROUND(avg_recency, 2) AS STRING), ' average days since order, ',
                CAST(ROUND(avg_frequency, 2) AS STRING), ' average total orders, ', 
                CAST(ROUND(avg_monetary, 2) AS STRING), ' average total spend, ',
                CAST(mode_recency_quartile AS STRING), ' most common quartile for days since order, ',
                CAST(mode_frequency_quartile AS STRING), ' most common quartile for total orders, ',
                CAST(mode_monetary_quartile AS STRING), ' most common quartile for total spend, ',
                'most common countries: ', first_country, ', ', second_country, ', ', third_country, ', ', fourth_country,
                '.'
            ) as shot

In [57]:
job = bq.query(query = query)
job.result()
result = job.to_dataframe()
result

,response
0,**Cluster 1: Loyal Customers**\n- Middle-aged...


In [58]:
print(result['response'].iloc[0])

 **Cluster 1: Loyal Customers**
- Middle-aged customers (average age: 42.71)
- Long-term customers (average days since order: 619.2)
- High repeat purchasers (average total orders: 5.35)
- Moderate spenders (average total spend: 334.06)
- Mostly from China, the United States, Brazil, and South Korea

**Cluster 2: Young and Frequent Buyers**
- Young customers (average age: 27.11)
- Relatively new customers (average days since order: 188.05)
- Frequent purchasers (average total orders: 2.89)
- Low spenders (average total spend: 158.87)
- Mostly from China, the United States, Brazil, and South Korea

**Cluster 3: High-Value Customers**
- Middle-aged customers (average age: 41.88)
- Moderate tenure customers (average days since order: 229.6)
- Very high repeat purchasers (average total orders: 25.33)
- Highest spenders (average total spend: 2321.59)
- Mostly from China, the United States, Brazil, and France

**Cluster 4: Big Spenders**
- Middle-aged customers (average age: 40.75)
- Moderat

---
## Predict Clusters Today


In [59]:
query = f"""
# 13
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_featuresToday` AS
WITH
    get_rfm AS (
        SELECT
            customer_id, age, gender, country,
            DATE_DIFF(DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH), MAX(order_date), DAY) AS recency,
            COUNT(order_id) AS frequency,
            SUM(revenue) AS monetary,
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_raw`
        GROUP BY customer_id, age, gender, country
    ),
    get_quartiles AS (
        SELECT
            customer_id,
            NTILE(4) OVER (ORDER BY recency ASC) AS recency_quartile,
            NTILE(4) OVER (ORDER BY frequency DESC) AS frequency_quartile,
            NTILE(4) OVER (ORDER BY monetary DESC) AS monetary_quartile,
        FROM get_rfm
    ),
    get_rfm_score AS (
        SELECT
            customer_id,
            recency_quartile + frequency_quartile + monetary_quartile AS rfm_score
        FROM get_quartiles
    )
SELECT
    CAST(rfm.customer_id AS STRING) AS customer_id,
    rfm.age,
    rfm.gender,
    rfm.country,
    rfm.recency,
    rfm.frequency,
    rfm.monetary,
    q.recency_quartile,
    q.frequency_quartile,
    q.monetary_quartile,
    rfms.rfm_score
FROM get_rfm as rfm
LEFT JOIN get_quartiles as q ON rfm.customer_id = q.customer_id
LEFT JOIN get_rfm_score as rfms ON rfm.customer_id = rfms.customer_id
"""

In [60]:
print(query)


# 13
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_featuresToday` AS
WITH
    get_rfm AS (
        SELECT
            customer_id, age, gender, country,
            DATE_DIFF(DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH), MAX(order_date), DAY) AS recency,
            COUNT(order_id) AS frequency,
            SUM(revenue) AS monetary,
        FROM `statmike-mlops-349915.bqml_2024.thelook_raw`
        GROUP BY customer_id, age, gender, country
    ),
    get_quartiles AS (
        SELECT
            customer_id,
            NTILE(4) OVER (ORDER BY recency ASC) AS recency_quartile,
            NTILE(4) OVER (ORDER BY frequency DESC) AS frequency_quartile,
            NTILE(4) OVER (ORDER BY monetary DESC) AS monetary_quartile,
        FROM get_rfm
    ),
    get_rfm_score AS (
        SELECT
            customer_id,
            recency_quartile + frequency_quartile + monetary_quartile AS rfm_score
        FROM get_quartiles
    )
SELECT
    CAST(rfm.customer_id AS STRING

In [61]:
job = bq.query(query = query)
job.result()

---
## Forecasting Revenue By Segment

In [62]:
query = f'''
# 14
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_forecast` AS
    WITH
        SEGMENTS AS (
            SELECT customer_id, centroid_id as cluster
            FROM ML.PREDICT (
                MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`,
                (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_featuresToday`)
            )
        )
    SELECT DATE_TRUNC(r.order_date, MONTH) as order_date, s.cluster, SUM(r.revenue) as total_revenue
    FROM SEGMENTS as s
    JOIN `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_raw` as r ON CAST(r.customer_id AS STRING) = s.customer_id
    GROUP BY s.cluster, order_date
    ORDER BY s.cluster, order_date
;
'''

In [63]:
print(query)


# 14
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_forecast` AS
    WITH
        SEGMENTS AS (
            SELECT customer_id, centroid_id as cluster
            FROM ML.PREDICT (
                MODEL `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`,
                (SELECT * FROM `statmike-mlops-349915.bqml_2024.thelook_featuresToday`)
            )
        )
    SELECT DATE_TRUNC(r.order_date, MONTH) as order_date, s.cluster, SUM(r.revenue) as total_revenue
    FROM SEGMENTS as s
    JOIN `statmike-mlops-349915.bqml_2024.thelook_raw` as r ON CAST(r.customer_id AS STRING) = s.customer_id
    GROUP BY s.cluster, order_date
    ORDER BY s.cluster, order_date
;



In [64]:
job = bq.query(query = query)
job.result()

In [65]:
query = f"""
# 15
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_arimaplus`
OPTIONS
    (model_type = 'ARIMA_PLUS',
    time_series_timestamp_col = 'order_date',
    time_series_data_col = 'total_revenue',
    time_series_id_col = 'cluster',
    data_frequency = 'MONTHLY',
    auto_arima_max_order = 5,
    holiday_region = ['CN', 'BR', 'KR', 'US'],
    horizon = 12
    #,forecast_limit_lower_bound = 0
    ) AS
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_forecast`
WHERE order_date < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
"""
job = bq.query(query)
job.result()
(job.ended-job.started).total_seconds()

16.915

In [66]:
print(query)


# 15
CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.thelook_arimaplus`
OPTIONS
    (model_type = 'ARIMA_PLUS',
    time_series_timestamp_col = 'order_date',
    time_series_data_col = 'total_revenue',
    time_series_id_col = 'cluster',
    data_frequency = 'MONTHLY',
    auto_arima_max_order = 5,
    holiday_region = ['CN', 'BR', 'KR', 'US'],
    horizon = 12
    #,forecast_limit_lower_bound = 0
    ) AS
SELECT *
FROM `statmike-mlops-349915.bqml_2024.thelook_forecast`
WHERE order_date < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)



In [67]:
query = f'''
# 16
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_forecastResult` AS
SELECT *, 
    DATE(DATE_TRUNC(time_series_timestamp, MONTH)) AS order_date
FROM ML.EXPLAIN_FORECAST(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_arimaplus`,
    STRUCT(12 AS horizon, 0.95 AS confidence_level))
ORDER BY cluster, order_date
'''
print(query)


# 16
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_forecastResult` AS
SELECT *, 
    DATE(DATE_TRUNC(time_series_timestamp, MONTH)) AS order_date
FROM ML.EXPLAIN_FORECAST(
    MODEL `statmike-mlops-349915.bqml_2024.thelook_arimaplus`,
    STRUCT(12 AS horizon, 0.95 AS confidence_level))
ORDER BY cluster, order_date



In [68]:
job = bq.query(query)
job.result()
(job.ended-job.started).total_seconds()

3.708


---
### SWITCH TO DATAFRAMES WORK IN BQ STUDIO

---

---
## Early Categorization of New Customers